In [ ]:
# Ensure a qsharp version of 1.17.0 or newer in this configuration when using qiskit>=2.0.

import pathlib

from ft_optimization_generate_data import (
    SINGLE_QUBIT_AND_CX_QISKIT_STDGATES,
    generate_data,
)
from pytket import OpType
from pytket.passes import (
    CliffordSimp,
    CommuteThroughMultis,
    EulerAngleReduction,
    FullPeepholeOptimise,
    KAKDecomposition,
    RemoveRedundancies,
    RepeatPass,
    SequencePass,
)
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary
from qiskit.transpiler.passes import BasisTranslator
from qiskit.transpiler.passes.optimization import (
    CommutationAnalysis,
    CommutativeCancellation,
    HoareOptimizer,
    Optimize1qGatesDecomposition,
    Optimize1qGatesSimpleCommutation,
    TemplateOptimization,
)

# Qiskit

In [ ]:
circuit_folder = "mqtbench"

benchmarks = [f.name.replace(".qasm", "") for f in pathlib.Path(circuit_folder).iterdir() if f.name.endswith(".qasm")]
benchmark_names = [name.split("_indep_qiskit_")[0] for name in benchmarks]
benchmarks.sort()

basis_gates = SINGLE_QUBIT_AND_CX_QISKIT_STDGATES

transpiler_passes = [
    [Optimize1qGatesDecomposition(basis=basis_gates), BasisTranslator(SessionEquivalenceLibrary, basis_gates)],
    [Optimize1qGatesSimpleCommutation(basis=basis_gates), BasisTranslator(SessionEquivalenceLibrary, basis_gates)],
    [
        CommutationAnalysis(),
        CommutativeCancellation(basis_gates=basis_gates),
        BasisTranslator(SessionEquivalenceLibrary, basis_gates),
    ],
    [HoareOptimizer(), BasisTranslator(SessionEquivalenceLibrary, basis_gates)],
    [TemplateOptimization(), BasisTranslator(SessionEquivalenceLibrary, basis_gates)],
]

In [ ]:
generate_data(
    "results_qiskit.xlsx",
    benchmarks,
    transpiler_passes,
    None,
    "qiskit",
)

# Tket

In [ ]:
circuit_folder = "mqtbench"

benchmarks = [f.name.replace(".qasm", "") for f in pathlib.Path(circuit_folder).iterdir() if f.name.endswith(".qasm")]
benchmark_names = [name.split("_indep_tket_")[0] for name in benchmarks]
benchmarks.sort()

transpiler_passes = [
    RemoveRedundancies(),
    CliffordSimp(),
    FullPeepholeOptimise(),
    EulerAngleReduction(OpType.Rx, OpType.Ry),
    KAKDecomposition(),
    RepeatPass(SequencePass([CommuteThroughMultis(), RemoveRedundancies()])),
]

transpiler_pass_names = [
    "RemoveRedundancies",
    "CliffordSimp",
    "FullPeepholeOptimise",
    "EulerAngleReduction",
    "KAKDecomposition",
    "RepeatPass",
]

In [ ]:
generate_data("results_tket.xlsx", benchmarks, transpiler_passes, transpiler_pass_names, "tket")